In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd
import re
import pickle
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm
chrome_options = Options()
import json
chrome_options.add_argument("--window-size=1920x1080")  # Set window size
#chrome_options.add_argument('--headless') 

In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

C:\Users\ntmduc\AppData\Local\Temp\ipykernel_12144\3910758376.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


In [10]:
def get_data_info(driver):
    res = {}
    for i in range(5):
        driver.execute_script(f"window.scrollTo(0,{500*i})")
        time.sleep(1)
    driver.execute_script("arguments[0].scrollIntoView();", driver.find_element("xpath", "//button[@class='comet-v2-btn comet-v2-btn-slim comet-v2-btn-large specification--btn--CXRSSZD comet-v2-btn-important']"))
    driver.execute_script(f"window.scrollBy(0,-{500});")
    driver.find_element("xpath", "//button[@class='comet-v2-btn comet-v2-btn-slim comet-v2-btn-large specification--btn--CXRSSZD comet-v2-btn-important']").click()
    time.sleep(2)
    nav_specification = driver.find_element("xpath", "//div[@id = 'nav-specification']")
    
    specification = nav_specification.find_elements("xpath", ".//div[@class = 'specification--group--GtBZQvv']")
    for sample in specification:
        current_tmp_save = {}
        group_title = sample.find_element("xpath", ".//div[@class='specification--groupTitle--snzN4gh']").text
        list_spec = sample.find_elements("xpath", ".//li[@class = 'specification--line--iUJOqof']")
        tmp = []
        for x in list_spec:
            tmp.extend(x.find_elements("xpath", ".//div[@class = 'specification--prop--RejitI8']"))
        
        for x in tmp:
            specification_title = x.find_element("xpath", ".//div[@class = 'specification--title--UbVeyic']").text
            specification_desc = x.find_element("xpath", ".//div[@class = 'specification--desc--Mz148Bl']").text
            current_tmp_save[specification_title] = specification_desc
        res[group_title] = current_tmp_save
    res["price"] = driver.find_element("xpath", ".//div[@class = 'price--original--qDQaH8V']").find_element("xpath", ".//span[@class = 'price--originalText--Zsc6sMv']").text
    return res

In [11]:
data_path = "save_link/page{}.json"
save_path = "data/page{}.csv"
error_link = "error.json"
for i in range(2, 31):
    res_df = []
    error = []
    file_path = data_path.format(i)
    print("Crawl page {} - path: {}".format(i, file_path))
    with open(file_path, 'r') as f:
        data = json.load(f)
        for path in tqdm(data):
            try:
                driver.get(path)
                res = get_data_info(driver)
                res_df.append(res)
                time.sleep(2)
            except:
                error.append(path)
    with open(save_path.format(i), "w") as f:
        json.dump(res_df, f)
    with open(error_link, 'a') as f:
        json.dump(error, f)        
    time.sleep(2)

Crawl page 1 - path: save_link/page1.json


100%|██████████| 60/60 [17:38<00:00, 17.65s/it]


Crawl page 2 - path: save_link/page2.json


 10%|█         | 6/60 [01:47<13:59, 15.55s/it]

'₫6,566,240'